# Solving PDEs on domians defined via multiple level sets

In order to solve a partial differential equation on a stationary domain defined by multiple level set functions, we proceed similarly to the tutorial [cutfem.ipynb](cutfem.ipynb), with a few key differences in order to deal with corners.

For simplicity we consider a Poisson problem on the unit square in order to show the differences between the single and multiple level set setting. 

The libraries are imported as usual:

In [ ]:
import netgen.gui
from ngsolve.internal import *

# Basic NGSolve things
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.solvers import PreconditionedRichardson as PreRic

# ngsxfem and the mlset convenience layer 
from xfem import *
from xfem.mlset import *

We wanto to solve the problem
\begin{align*}
    -\Delta u &= f &&\text{in }\Omega\\
    u &= 0 &&\text{on } \partial\Omega
\end{align*}
where the domain $\Omega\subset\widetilde\Omega$ is a subset of the meshed background domain $\widetilde\Omega$.

In [ ]:
geo = SplineGeometry()
geo.AddRectangle((-0.2, -0.2), (1.2, 1.2), 
                 bcs=("bottom", "right", "top", "left"))
mesh = Mesh(geo.GenerateMesh(maxh=0.1))

We define the domain as the set $\Omega := \bigcap_i \{\phi_i < 0\}$ with $\{\phi_i\} = \{ -y, x - 1, y - 1, -x \}$ and use a `DomainTypeArray` as a container for this set and to generate the boundary.

In [ ]:
level_sets = [-y, x - 1, y - 1, -x]
level_sets_p1 = tuple(GridFunction(H1(mesh, order=1)) for i in range(len(level_sets)))
for i, lsetp1 in enumerate(level_sets_p1):
    InterpolateToP1(level_sets[i], lsetp1)
    Draw(lsetp1, mesh, "lsetp1_{}".format(i))
visoptions.mminval = 0.0; visoptions.mmaxval = 0.0
visoptions.deformation = 0; visoptions.autoscale = 0

square = DomainTypeArray((NEG, NEG, NEG, NEG))
boundary = square.Boundary()

With the domain and the boundary as `DomainTypeArray`s at hand, we generate the necessary dictionaries needed for integration later.

In [ ]:
lset_dom_inner = {"levelset": level_sets_p1, "domain_type": square}
lsets_bnd = {dtt: {"levelset": level_sets_p1, "domain_type": dtt} 
             for dtt in boundary}

Finite Element spaces are defined as usual:

In [ ]:
k=3

V = H1(mesh, order=k, dgjumps=True)
gfu = GridFunction(V)

## Multiple level set cut-information


As before, we use standard FE spaces restricted to the active part of the mesh. In order to handle multiple level sets, we need the `MultiLevelsetCutInfo` class:

In [ ]:
mlci = MultiLevelsetCutInfo(mesh, level_sets_p1)

### Marking the correct elements

Marking the correct active elements needs to be handled differently in the multipe level set setting, since elements of the type `(IF,IF)` are sometimes not part of the active domain, as can be seen in the following image:
<center><img src="graphics/mlset_cuts.png" alt="" width="500"/></center>

In order to select the correct active elements, the `MultiLevelsetCutInfo` class has a member function `GetElementsWithContribution()`. This takes in a `tuple(ENUM)`, a list of such tuples or a `DomainTypeArray`

In [ ]:
els_hasneg = mlci.GetElementsWithContribution(square)
els_if = mlci.GetElementsWithContribution(boundary)

Draw(BitArrayCF(els_hasneg), mesh, "els_hasneg")
Draw(BitArrayCF(els_if), mesh, "els_if")

To deal with arbitraty mesh-interface cuts we use ghost-penalty stabilisation. The elements needed for this can be obtained in the same manner as before and similarly, the free degrees of freedom can also be determined as before:

In [ ]:
facets_gp = GetFacetsWithNeighborTypes(mesh, a=els_hasneg, b=els_if, 
                                       use_and=True)

freedofs = GetDofsOfElements(V, els_hasneg) & V.FreeDofs()

We implement the homogeneous Dirichlet boundary conditions using Nitsche's trick. As each boundaty section has a different unit-normal vector, we have to decompose the Nitsche term into the different segments. Each will be added as a separate weak form, so we also require element markers for each side seperately.

Since the easient and most flexible nomenclature for each segment is the tuple of `DOMAIN_TYPE`s, we store these element markers in a dictionary using the tuple as the key:

In [ ]:
els_if_singe = {dtt: BitArray(mesh.ne) for dtt in boundary}

for i, (dtt, els_bnd) in enumerate(els_if_singe.items()):
    els_bnd[:] = False
    els_bnd |= mlci.GetElementsWithContribution(dtt)
    Draw(BitArrayCF(els_bnd), mesh, "els_if_singe"+str(i))

## Defining the weak form

As in any other unfitted problem, we define some parameters and get the trial and test functions and mesh-size coefficient functions as usual.

In [ ]:
gamma_n = 10
gamma_s = 0.5

u, v = V.TnT()
h = specialcf.mesh_size

For the normal vectors, we use the `DomainTypeArray.GetOuterNormals()` functionality, see [mlset_basic.ipynb](mlset_basic.ipynb)

In [ ]:
normals = square.GetOuterNormals(level_sets_p1)

The diffusion and ghost-penalty forms can then be defined as usual

In [ ]:
diffusion = InnerProduct(Grad(u), Grad(v))
ghost_penalty = gamma_s / (h**2) * (u - u.Other()) * (v - v.Other())

while we define the Nitsche term as a function of the normal vector

In [ ]:
def nitsche(n):
    return - InnerProduct(Grad(u) * n, v) \
           - InnerProduct(Grad(v) * n, u) \
           + (gamma_n * k * k / h) * InnerProduct(u, v)

We want the exact solution to be $u = 16 x (1 - x) y (1 - y)$. We thefore also define the forcing term as

In [ ]:
forcing = 32 * (y * (1 - y) + x * (1 - x)) * v

### Integrators

As we have an unfitted problem, we use a `RestrictedBilinearForm`

In [ ]:
a = RestrictedBilinearForm(V, element_restriction=els_hasneg, 
                           facet_restriction=facets_gp, check_unused=False)

The `diffusion` and `ghost_penalty` forms can then be added normally

In [ ]:
a += SymbolicBFI(lset_dom_inner, form=diffusion, 
                 definedonelements=els_hasneg)
a += SymbolicFacetPatchBFI(form=ghost_penalty, skeleton=False, 
                           definedonelements=facets_gp)

For the Nitsche terms, we loop over the normals dictionary. This gives us acess to both the boundary domain-tuples and the normal vectors:

In [ ]:
for bnd, n in normals.items():
    a += SymbolicBFI(lsets_bnd[bnd], form=nitsche(n), 
                     definedonelements=els_if_singe[bnd])

For the right-hand side, we do not have anything special

In [ ]:
f = LinearForm(V)
f += SymbolicLFI(lset_dom_inner, form=forcing)

## Solving the system

Now the system has been set up, we can assemble the systrem and sove in the ususal manner

In [ ]:
f.Assemble()
a.Assemble()
inv = a.mat.Inverse(freedofs=freedofs, inverse="sparsecholesky")

gfu.vec.data = PreRic(a=a, rhs=f.vec, pre=inv, freedofs=freedofs)

We can now visualise the solution:

In [ ]:
u_ex = 16 * x * (1 - x) * y * (1 - y)
grad_u_ex = CoefficientFunction((16 * (1 - 2 * x) * y * (1 - y), 
                                 16 * x * (1 - x) * (1 - 2 * y)))
Draw(gfu, mesh, "solution")
Draw((gfu - 16 * x * (1 - x) * y * (1 - y)), mesh, "err")

And compute the error

In [ ]:
err_l2 = sqrt(Integrate(lset_dom_inner, 
                        cf=InnerProduct(gfu - u_ex, gfu - u_ex),
                        mesh=mesh, order=2 * k))
err_h1 = sqrt(Integrate(lset_dom_inner, 
                        cf=InnerProduct(Grad(gfu) - grad_u_ex, Grad(gfu) - grad_u_ex),
                        mesh=mesh, order=2 * (k - 1)))
print("L2 error = {:1.3e}".format(err_l2))
print("H1 error = {:1.3e}".format(err_h1))